## Script to load in a processed seurat object and run pySCENIC

To get your processed Seurat object into adata format you can use conversion functions, such as:
```r
#load custom functions
source("/pl/active/CSUClinHeme/users/dylan/repos/scrna-seq/analysis-code/customFunctions_Seuratv5.R")

#load in processed R data and subset on cells of interest
seu.obj <- readRDS("../output/s3/allCells_clean_highRes_integrated.harmony_res1.6_dims45_dist0.1_neigh10_S3.rds")


#stash new idents
seu.obj.sub <- subset(seu.obj, subset = clusterID2_integrated.harmony %in% c(3,17,30))

cnts <- seu.obj.sub@assays$RNA$counts
cnts <- orthogene::convert_orthologs(gene_df = cnts,
                                        gene_input = "rownames", 
                                        gene_output = "rownames", 
                                        input_species = "dog",
                                        output_species = "human",
                                        non121_strategy = "drop_both_species") 
rownames(cnts) <- unname(rownames(cnts))

seu.obj <- CreateSeuratObject(cnts, project = "humanConvert", assay = "RNA",
                                  min.cells = 0, min.features = 0, names.field = 1,
                                  names.delim = "_", meta.data = seu.obj.sub@meta.data)

#covert back to an older Seurat object for compatiability with 
seu.obj[["RNA"]] <- as(object = seu.obj[["RNA"]], Class = "Assay")
seu.obj <- NormalizeData(seu.obj)

SaveH5Seurat(seu.obj, filename = "../output/s3/myeloid_branch1_hu.h5Seurat", verbose = TRUE, overwrite = T)
Convert("../output/s3/myeloid_branch1_hu.h5Seurat", dest = "h5ad", overwrite = T)

```

### Note: TF files were obtained from https://resources.aertslab.org/cistarget/

In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from matplotlib.pyplot import rc_context
from MulticoreTSNE import MulticoreTSNE as TSNE
from pyscenic.export import export2loom, add_scenic_metadata

import seaborn as sns
import matplotlib.pyplot as plt
import scipy 

import glob

import json
import zlib
import base64

In [24]:
FIGURES_FOLDERNAME="../output/scenic/"
def savesvg(fname: str, fig, folder: str=FIGURES_FOLDERNAME) -> None:
    """
    Save figure as vector-based SVG image format.
    """
    fig.tight_layout()
    fig.savefig(os.path.join(folder, fname), format='svg')

In [25]:
#load in the data
adata = sc.read_h5ad("../output/s3/myeloid_branch1_hu.h5ad")
adata

AnnData object with n_obs × n_vars = 2084 × 11606
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.hbm', 'percent.ppbp', 'RNA_snn_res.0.1', 'seurat_clusters', 'int.clusID', 'pANN_0.25_0.01_274', 'DF.classifications_0.25_0.01_274', 'doublet', 'S.Score', 'G2M.Score', 'Phase', 'clusters', 'pANN_0.25_0.19_108', 'DF.classifications_0.25_0.19_108', 'pANN_0.25_0.02_98', 'DF.classifications_0.25_0.02_98', 'pANN_0.25_0.11_115', 'DF.classifications_0.25_0.11_115', 'pANN_0.25_0.005_95', 'DF.classifications_0.25_0.005_95', 'unintegrated_clusters', 'RNA_snn_res.0.6', 'clusterID_integrated.cca', 'clusterID_integrated.harmony', 'clusterID_integrated.joint', 'clusterID_integrated.rcpa', 'name', 'colz', 'cellSource', 'minorIdent', 'majorID', 'major_clusID', 'pseudotime', 'ptime', 'RNA_snn_res.1.6', 'clusterID2_integrated.harmony'
    var: 'features'

In [27]:
#convert from numeric to string
anno = adata.obs
anno['clusterID2_integrated.harmony'] = anno['clusterID2_integrated.harmony'].astype(str)
adata.obs = anno

In [28]:
#convert index to gene symbol
tempAdata = adata.raw.to_adata()
tempAdata.var_names = adata.var['features']
adata.raw = tempAdata

In [ ]:
#cant run b/c UMAP coords weren't brought over
# sc.pl.umap(adata, color=['IGHM','majorID_sub2','name'], use_raw = False)

In [29]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( "../output/s3/myeloid_branch1_hu.loom", adata.X.transpose(), row_attrs, col_attrs)

The easiest way to run the code is to use a sbatch script, so generate a script with:

In [30]:
!echo "#!/usr/bin/env bash" > cute_pyScenic_grn.sbatch
!echo "#SBATCH --job-name=pySCENIC_grn" >> cute_pyScenic_grn.sbatch

!echo "#SBATCH --nodes=1" >> cute_pyScenic_grn.sbatch
!echo "#SBATCH --ntasks=12" >> cute_pyScenic_grn.sbatch
!echo "#SBATCH --time=00:30:00" >> cute_pyScenic_grn.sbatch

!echo "#SBATCH --partition=atesting" >> cute_pyScenic_grn.sbatch
!echo "#SBATCH --qos=normal" >> cute_pyScenic_grn.sbatch

!echo "#SBATCH --mail-type=END" >> cute_pyScenic_grn.sbatch
!echo "#SBATCH --mail-user=dyammons@colostate.edu" >> cute_pyScenic_grn.sbatch
!echo "#SBATCH --output=pySCENIC_grn-%j.log" >> cute_pyScenic_grn.sbatch

!echo "" >> cute_pyScenic_grn.sbatch

!echo "module purge" >> cute_pyScenic_grn.sbatch

!echo "" >> cute_pyScenic_grn.sbatch

!echo "singularity exec -B $PWD/../ ../software/aertslab-pyscenic-scanpy-0.12.1-1.9.1.sif pyscenic grn \\" >> cute_pyScenic_grn.sbatch
!echo "--num_workers \$SLURM_NTASKS \\" >> cute_pyScenic_grn.sbatch
!echo "-o ../output/scenic/adj.csv \\" >> cute_pyScenic_grn.sbatch
!echo "../output/s3/myeloid_branch1_hu.loom \\" >> cute_pyScenic_grn.sbatch
!echo "./metaData/allTFs_hg38.txt" >> cute_pyScenic_grn.sbatch

In [31]:
#submit the job
!sbatch cute_pyScenic_grn.sbatch

Submitted batch job 4645519


In [32]:
!echo "#!/usr/bin/env bash" > cute_pyScenic_ctx.sbatch
!echo "#SBATCH --job-name=pySCENIC_ctx" >> cute_pyScenic_ctx.sbatch

!echo "#SBATCH --nodes=1" >> cute_pyScenic_ctx.sbatch
!echo "#SBATCH --ntasks=10" >> cute_pyScenic_ctx.sbatch
!echo "#SBATCH --time=00:20:00" >> cute_pyScenic_ctx.sbatch

!echo "#SBATCH --partition=atesting" >> cute_pyScenic_ctx.sbatch
!echo "#SBATCH --qos=normal" >> cute_pyScenic_ctx.sbatch

!echo "#SBATCH --mail-type=END" >> cute_pyScenic_ctx.sbatch
!echo "#SBATCH --mail-user=dyammons@colostate.edu" >> cute_pyScenic_ctx.sbatch
!echo "#SBATCH --output=pySCENIC_ctx-%j.log" >> cute_pyScenic_ctx.sbatch

!echo "" >> cute_pyScenic_ctx.sbatch

!echo "module purge" >> cute_pyScenic_ctx.sbatch

!echo "" >> cute_pyScenic_ctx.sbatch

!echo "singularity exec -B $PWD/../ ../software/aertslab-pyscenic-scanpy-0.12.1-1.9.1.sif pyscenic ctx \\" >> cute_pyScenic_ctx.sbatch
!echo "../output/scenic/adj.csv \\" >> cute_pyScenic_ctx.sbatch
!echo "./metaData/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather ./metaData/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \\" >> cute_pyScenic_ctx.sbatch
!echo "--annotations_fname ./metaData/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl \\" >> cute_pyScenic_ctx.sbatch
!echo "--expression_mtx_fname ../output/s3/myeloid_branch1_hu.loom \\" >> cute_pyScenic_ctx.sbatch
!echo "--output ../output/scenic/reg.csv \\" >> cute_pyScenic_ctx.sbatch
!echo "--mask_dropouts \\" >> cute_pyScenic_ctx.sbatch
!echo "--num_workers \$SLURM_NTASKS" >> cute_pyScenic_ctx.sbatch


In [33]:
#submit the job; took 8 min....
!sbatch cute_pyScenic_ctx.sbatch

Submitted batch job 4647353


In [34]:
!echo "#!/usr/bin/env bash" > cute_pyScenic_aucell.sbatch
!echo "#SBATCH --job-name=pySCENIC_auc" >> cute_pyScenic_aucell.sbatch

!echo "#SBATCH --nodes=1" >> cute_pyScenic_aucell.sbatch
!echo "#SBATCH --ntasks=4" >> cute_pyScenic_aucell.sbatch
!echo "#SBATCH --time=0:10:00" >> cute_pyScenic_aucell.sbatch

!echo "#SBATCH --partition=atesting" >> cute_pyScenic_aucell.sbatch
!echo "#SBATCH --qos=normal" >> cute_pyScenic_aucell.sbatch

!echo "#SBATCH --mail-type=END" >> cute_pyScenic_aucell.sbatch
!echo "#SBATCH --mail-user=dyammons@colostate.edu" >> cute_pyScenic_aucell.sbatch
!echo "#SBATCH --output=pySCENIC_auc-%j.log" >> cute_pyScenic_aucell.sbatch

!echo "" >> cute_pyScenic_aucell.sbatch
!echo "module purge" >> cute_pyScenic_aucell.sbatch

!echo "" >> cute_pyScenic_aucell.sbatch

!echo "singularity exec -B $PWD/../ ../software/aertslab-pyscenic-scanpy-0.12.1-1.9.1.sif pyscenic aucell \\" >> cute_pyScenic_aucell.sbatch
!echo "../output/s3/myeloid_branch1_hu.loom \\" >> cute_pyScenic_aucell.sbatch
!echo "../output/scenic/reg.csv \\" >> cute_pyScenic_aucell.sbatch
!echo "--output ../output/scenic/myeloid_branch1_hu_output.loom \\" >> cute_pyScenic_aucell.sbatch
!echo "--num_workers \$SLURM_NTASKS" >> cute_pyScenic_aucell.sbatch


In [35]:
#took 2 min....
!sbatch cute_pyScenic_aucell.sbatch

Submitted batch job 4648003


In [36]:
# collect SCENIC AUCell output
# lf = lp.connect("./output/scenic/dc_scenic_output_231023.loom", mode='r+', validate=False )
lf = lp.connect("../output/scenic/myeloid_branch1_hu_output.loom", mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [37]:
auc_mtx

,AHR(+),ARID3A(+),ARNT(+),ATF1(+),ATF2(+),ATF3(+),ATF5(+),BACH1(+),BACH2(+),BCL6(+),...,ZNF768(+),ZNF770(+),ZNF774(+),ZNF782(+),ZNF784(+),ZNF792(+),ZNF8(+),ZSCAN18(+),ZSCAN22(+),ZXDC(+)
BM154801_AAACGAAAGCAACAAT-1,0.006149,0.348276,0.001826,0.000000,0.000840,0.014325,0.0,0.030948,0.0,0.010875,...,0.014368,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.045211,0.012835
BM154801_AAAGAACAGGTAGACC-1,0.062858,0.000000,0.000000,0.021552,0.000000,0.010437,0.0,0.015216,0.0,0.038581,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.015038,0.000000
BM154801_AAAGAACGTAGGTCAG-1,0.004196,0.043966,0.000000,0.000000,0.000818,0.013023,0.0,0.000000,0.0,0.017666,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.056801,0.000000
BM154801_AAAGGGCGTCTGTCCT-1,0.003303,0.000000,0.004158,0.000000,0.000000,0.005209,0.0,0.000000,0.0,0.010438,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.025383,0.005090
BM154801_AAAGTCCAGTTGTAGA-1,0.006855,0.000000,0.002738,0.000000,0.000000,0.008731,0.0,0.008060,0.0,0.004456,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.022126,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C34165634_TTTCGATAGTCTGCAT-1,0.007395,0.000000,0.000000,0.000000,0.000000,0.004604,0.0,0.000000,0.0,0.018302,...,0.000000,0.0,0.0,0.026293,0.0,0.0,0.0,0.0,0.029885,0.004269
C34165634_TTTGATCAGAGAAGGT-1,0.000000,0.000000,0.000000,0.004119,0.001459,0.003173,0.0,0.000259,0.0,0.018939,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.027682,0.003914
C34165634_TTTGGAGCACTCTGCT-1,0.002098,0.000000,0.000000,0.000000,0.000000,0.007062,0.0,0.005129,0.0,0.008647,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.010632,0.015161
C34165634_TTTGGAGGTAAGGCCA-1,0.006128,0.000000,0.000000,0.004502,0.003360,0.007465,0.0,0.001164,0.0,0.005093,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.017146,0.004789


In [18]:
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
import matplotlib.pyplot as plt
# from adjustText import adjust_text
import seaborn as sns
from pyscenic.binarization import binarize

In [38]:
cellAnnot = pd.concat(
    [
        pd.DataFrame( adata.obs.index.tolist(), index=adata.obs['clusterID2_integrated.harmony'].tolist() )
    ],
    axis=1
)
cellAnnot.columns = [
 'clusterID2_integrated.harmony']


In [39]:
rss = regulon_specificity_scores(auc_mtx, adata.obs['clusterID2_integrated.harmony'])
rss.head()

,AHR(+),ARID3A(+),ARNT(+),ATF1(+),ATF2(+),ATF3(+),ATF5(+),BACH1(+),BACH2(+),BCL6(+),...,ZNF768(+),ZNF770(+),ZNF774(+),ZNF782(+),ZNF784(+),ZNF792(+),ZNF8(+),ZSCAN18(+),ZSCAN22(+),ZXDC(+)
17,0.345594,0.531878,0.278413,0.255813,0.276130,0.361036,0.182742,0.426544,0.194784,0.382647,...,0.275423,0.191327,0.173165,0.212178,0.187939,NaN,0.197946,0.275173,0.385119,0.336470
30,0.431298,0.184048,0.257456,0.254651,0.268700,0.285789,0.176422,0.333909,0.206036,0.300222,...,0.212837,0.175888,0.167445,0.181953,0.191452,NaN,0.216853,0.212957,0.314162,0.245767
3,0.401027,0.244819,0.250585,0.283455,0.410835,0.558180,0.182847,0.340268,0.187611,0.567580,...,0.341794,0.172368,0.182956,0.301025,0.185764,NaN,0.224002,0.235091,0.458189,0.534193


In [40]:
rss.columns.values

array(['AHR(+)', 'ARID3A(+)', 'ARNT(+)', 'ATF1(+)', 'ATF2(+)', 'ATF3(+)',
       'ATF5(+)', 'BACH1(+)', 'BACH2(+)', 'BCL6(+)', 'BHLHE40(+)',
       'BRCA1(+)', 'CBFB(+)', 'CCNT2(+)', 'CDC5L(+)', 'CEBPB(+)',
       'CEBPE(+)', 'CEBPG(+)', 'CHD2(+)', 'CLOCK(+)', 'CNOT3(+)',
       'CREB1(+)', 'CUX1(+)', 'DBP(+)', 'DDIT3(+)', 'DEAF1(+)', 'E2F1(+)',
       'E2F2(+)', 'E2F3(+)', 'E2F6(+)', 'E2F7(+)', 'E2F8(+)', 'EBF1(+)',
       'ELF1(+)', 'ELF2(+)', 'ELF4(+)', 'ELK1(+)', 'ELK3(+)', 'ELK4(+)',
       'EP300(+)', 'ERF(+)', 'ERG(+)', 'ESRRA(+)', 'ETS1(+)', 'ETS2(+)',
       'ETV3(+)', 'ETV5(+)', 'ETV6(+)', 'ETV7(+)', 'FLI1(+)', 'FOS(+)',
       'FOSB(+)', 'FOSL1(+)', 'FOSL2(+)', 'FOXJ2(+)', 'FOXK1(+)',
       'FOXM1(+)', 'FOXN2(+)', 'FOXO1(+)', 'FOXO4(+)', 'GABPA(+)',
       'GABPB1(+)', 'GATA1(+)', 'GATA2(+)', 'GATA3(+)', 'GFI1(+)',
       'GFI1B(+)', 'GLIS1(+)', 'GMEB1(+)', 'GTF2IRD1(+)', 'GTF3A(+)',
       'HDAC2(+)', 'HIF1A(+)', 'HLTF(+)', 'HMG20B(+)', 'HNF4A(+)',
       'HOXA10(+)', 'HOX

In [41]:
rss = rss.rename(index={'0' : '17',
                        '1' : '30',
                        '2' : '3'})

In [42]:
pd.DataFrame.to_csv(rss, "../output/scenic/myeloid_branch1_hu_rss.csv")


```r
library(tidyverse)
library(ggrepel)
library(patchwork)

df <- read.csv("../output/scenic/myeloid_branch1_hu_rss.csv", row.names = 1)
colnames(df) <- gsub('\\.\\.\\.',"",colnames(df))
df.all <- t(df) %>% as.data.frame() %>% rownames_to_column() %>% na.omit()

labCut = 10
pi <- lapply(2:ncol(df.all), function(x){
    #prep data
    celltype <- colnames(df.all)[x]
    df <- df.all[ ,c("rowname",celltype)] 
    colnames(df) <- c("gene", "clus")
    df <- df %>% arrange(desc(clus))
    df[ ,"gene"] <- factor(df[ ,"gene"], levels = df[ ,"gene"])
    df <- df %>% mutate(colz = ifelse(row_number() < labCut+1, "#FF6961", "lightblue"),
                        labz = ifelse(row_number() < labCut+1, as.character(gene), NA))
    
    #plot the data
    p <- ggplot(df, aes(x = gene, y = clus, label = labz)) +
    geom_point(colour = df$colz, size = 2) + 
    geom_text_repel(
        force = 0.01,
        nudge_x = nrow(df)*0.5,
        direction = "y",
        seed = 42, 
        box.padding = 0.25,
        #hjust = 0,
        #segment.size = 0.2, 
        size = 4, 
        color = df$colz,
        max.iter = 100000000,
        max.overlaps = 10
    ) + 
    labs(
        title = celltype,
        x = "Regulon",
        y = "rss"
    ) + 
    theme_classic() + 
    theme(
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank(),
        panel.border = element_blank(),
        plot.margin = unit(c(10, 10, 10, 10), "pt"),
        panel.background = element_rect(fill = "transparent",colour = NA),
        plot.background = element_rect(fill = "transparent",colour = NA),
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank()
    ) + 
    scale_y_continuous(
        breaks = function(y) {
            seq(floor(min(y, digits = 1)), 
            ceiling(max(y, digits = 1)), 
            by = 0.1)
        }
    ) + coord_cartesian(expand = TRUE, clip = "off")

    #return/save plot
    ggsave(paste0("../output/scenic/", gsub(" ", "_", celltype), "_rss.png"), height = 5, width = 3)
    return(p)
})


p <- Reduce( `+`, pi) + plot_layout(ncol = 3)
ggsave(plot = p, "../output/scenic/myeloid_branch1_rss.png", height = 4, width = 6)

```